# BottleNeck fine wine spirit

Projet 5 - Data Analyst

GIANNESINI Baptiste

# Table des matières <a class="anchor" id="chapter0"></a> 


*[Résumé de la mission](#chapter1)

*[Import des modules et packages](#chapter2)

*[Import des données](#chapter3)

*[exploration préliminaire et nettoyage de données](#chapter4)

**[données de la DF "Liaison"](#chapter5)

**[données de la DF "ERP"](#chapter6)

**[données de la DF "web"](#chapter7)

*[variables et fonctions](#chapter8)

*[Analyse et mise en forme des données](#chapter9)

*[Premières statistiques globales](#chapter10)

*[Résolution des problèmes énoncés](#chapter11)

*[rapprochement des bases](#chapter12)

*[chiffre d'affaire total en ligne](#chapter13)

*[chiffre d'affaire par produit](#chapter14)

*[recherche de valeurs anormales](#chapter15)

*[autres reflexions](#chapter16)

*[des invendus, pourquoi?](#chapter17)

*[gestion des stocks](#chapter18)



#  1 Résumé de la mission <a class="anchor" id="chapter1"></a>

Data Analyst freelance chez BottleNeck, un marchand de vin prestigieux, je dois composer avec un système d'analyses de ventes qualifié d'artisanal par le manager.
On me demande:
    1. de rapprocher un ERP des références produit avec une table issue de l'outil cms contenant les informations à propos de ces produits.
    2. calculer le chiffre d'affaires par produit
    3. calculer le total du chiffre d'affaire réalisé en ligne.
    4. rechercher d'éventuelles valeurs aberrantes dans les prix des produits, d'en faire une liste et une représentation graphique le cas échéant.



# 2 import des modules et packages <a class="anchor" id="chapter2"></a>

Pandas, numpy, matplotlib, plotly express et pandas_profiling

In [1]:
# imports modules et packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import plotly.express as px
from pandas_profiling import ProfileReport
import plotly.graph_objects as go



# 3 import des données <a class="anchor" id="chapter3"></a>

| Format | Nom     | contenu              | source |  
|--------|---------|----------------------|--------|  
| xlsx   | erp     | tarif/stock produits | fourni |  
| xlsx   | web     | description produits | fourni |  
| xlsx   | liaison | liaison erp/web      | fourni |  


In [2]:
# Import des données
# Fournies avec le projet

erp = pd.read_excel('erp.xlsx')
liaison = pd.read_excel('liaison.xlsx')
web = pd.read_excel('web.xlsx')


# 4 exploration préliminaire et nettoyage de données <a class="anchor" id="chapter4"></a>
Nous cherchons à connaitre la forme et le contenu de chaque dataframe afin de préparer au mieux leur fusion ultérieure.





## 5 Données DF "liaison" <a class="anchor" id="chapter5"></a>

In [3]:
#on regarde à quoi ressemblent les données de liaison:

print(liaison.shape)
liaison.head()


(825, 2)


,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815



Nous savons donc que la DF liaison contient 825 lignes et 2 colonnes appellées "product_id" et "id_web".

Ces deux colonnes faisant la jonction pour chaque produit du "product_id" correspondant a "product_id" dans la DF erp, et "id_web" correspondant a "sku" dans la DF web.



## 6 Données DF "erp"<a class="anchor" id="chapter6"></a>


In [4]:
#on regarde à quoi ressemblent les données de erp:
print(erp.shape)
erp.head()

(825, 5)


,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


Nous savons donc que la DF liaison contient 825 lignes et 5 colonnes appellées "product_id", "onsale_web", "price", "stock_quantity" et "stock-status".
Les colonnes qui nous interresseront particulièrement sont donc:
    1. product_id qui à sa correspondance dans la DF Liaison
    2. onsale_web un booleen indiquant si le produit est vendu en ligne
    3. price toujours utile pour calculer un CA.

Les colonnes correspondant au stock (quantité et disponibilité) ne sont pas nécessaires pour répondre aux questions posées, mais nous les conserverons tout de même.

## premières statistiques tirées de la DF erp

In [5]:
# On analyse le profileReport de erp

profileErp = ProfileReport(erp, title="Pandas Profiling Report")
profileErp.to_widgets()

Generate report structure: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it]



1. L'onglet Overview nous permet de vérifier qu'il n'y a pas de champs à la valeur NaN, ou de ligne dupliquées.


2. l'onglet Variables chaque variable confirme/nous informe que:

    1. product_id ne contient que des valeurs distinctes
    2. onsale_web est bien un booleen
    3. price ne contient que des valeurs positives comprises entre 5.2 et 225 avec une moyenne de 32.42
    4. stock_quantity ne contient que des valeurs positives comprises entre 0 et 578 avec une moyenne de 26.5
    5. stock-status est un booleen qui nous permet de savoir que 638 produits sont en stock et 187 ne sont plus en stock.


3. L'onglet Interactions nous donne un premier apercu graphique :

    1. du prix en fonction de l'id produit
    2. de la quantité de stock en fonction de l'id produit
    3. de la quantité de stock en fonction du prix

4. L'onglet missing values nous confirme que nos données ne contiennent pas de NaN


5. L'onglet Sample nous donne les premières et dernières lignes de la DF.



## 7 Données DF "web"<a class="anchor" id="chapter7"></a>

In [6]:
#on regarde à quoi ressemblent les données de web:

print(web.shape)
web.head()

(1513, 28)


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,bon-cadeau-25-euros,0,0,0,0.0,10.0,taxable,NaN,1.0,2018-06-01 13:53:46,...,bon-cadeau-de-25-euros,2018-06-01 14:13:57,2018-06-01 12:13:57,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
1,15298,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-08 12:58:52,...,pierre-jean-villa-saint-joseph-preface-2018,2019-12-30 09:30:29,2019-12-30 08:30:29,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
2,15296,0,0,0,0.0,0.0,taxable,NaN,2.0,2018-02-08 13:49:41,...,pierre-jean-villa-saint-joseph-tilde-2017,2019-12-21 09:00:17,2019-12-21 08:00:17,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,15300,0,0,0,0.0,0.0,taxable,NaN,2.0,2018-02-08 14:08:36,...,pierre-jean-villa-croze-hermitage-accroche-coe...,2020-06-26 18:15:03,2020-06-26 16:15:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,19814,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-09 14:01:05,...,pierre-jean-villa-igp-gamine-2018,2020-01-04 16:36:01,2020-01-04 15:36:01,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0


Nous savons donc maintenant que la DF web contient 1513 lignes et 28 colonnes.

Plusieurs constats sont a faires:

    1. nous attendions environ le même nombre de lignes que pour les DF précédentes soit 825, il est donc probable que des lignes soient dupliquées.

    2. nous pouvons constater dans les premières lignes de la DF que nous avons affiché au dessus, que certaines lignes sont en doublon (par exemple les lignes a l'index 1 et 2 correspondent a un produit dont le sku est identique)

    3. beaucoup de colonnes correspondent à des informations inutiles.

Il conviendra donc ici de procéder à une élimination des doublons et des colonnes inutiles.

In [7]:
# On commence par compter le nombre de produits uniques référencés dans la DF

web.sku.nunique()

714

714 références uniques, pour 825 références attendues cela signifie que l'ensemble des produits listés dans les DF erp et liaison ne sont pas commercialisés en ligne, nous le vérifieront plus tard.

In [8]:
# On observe l'ensemble des données contenue dans la DF web pour un seul produit 

webunique = web[web['sku'] == 15298]
webunique

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
1,15298,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-08 12:58:52,...,pierre-jean-villa-saint-joseph-preface-2018,2019-12-30 09:30:29,2019-12-30 08:30:29,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
799,15298,0,0,0,0.0,6.0,NaN,NaN,2.0,2018-02-08 12:58:52,...,pierre-jean-villa-saint-joseph-preface-2018,2019-12-30 09:30:29,2019-12-30 08:30:29,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


La DF web est issue du CMS du site web.

Nous constatons que les produits sont référencés dans la DF web au travers de deux lignes.

en observant la colonne post_type, on comprend que l'une des deux lignes correspond au texte descriptif du produit quand l'autre correspond aux illustrations du produit sur le site.

c'est donc une colonne adéquate pour éliminer les doublons en ne gardant qu'un seul type de ligne. 

Pour ce qui est des colonnes, nous allons supprimer tout ce dont nous ignorons la signification ou dont nous n'avons pas l'utilité.

Nous conserverons donc les colonnes:

    1. "sku" que nous renommerons en "id_web" pour faciliter la lecture et la jointure avec la df liaison.

    2. "total_sales" dont nous aurons besoin pour les calculs de CA qui nous ont été demandés.

    3. "post_name" que nous renommerons en "product_name" pour connaitre le nom des produits

    4. "post_type" afin de pouvoir trier les doublons



In [9]:
# nettoyage et mise en forme des données df web

# changement nom de colonnes
web.rename(columns={
    "sku": "id_web",
    "post_name" : "product_name"
    }, inplace=True)

# drop de colonnes
web = web.drop(["virtual", "downloadable", "rating_count","average_rating",  "tax_class", "post_author", "post_date","post_date_gmt", "post_content", "post_title", "post_excerpt", "post_status", "comment_status", "ping_status", "post_password", "post_modified", "post_modified_gmt", "post_content_filtered", "post_parent", "guid", "menu_order", "tax_status", "post_mime_type", "comment_count" ], axis=1)
erp = erp.drop(["stock_status"], axis=1)

# drop de lignes
web = web[web.post_type == 'product']


Nous devrions maintenant avoir une DF de 714 lignes et 4 colonnes:

In [10]:
print(web.shape)
web.head()

(716, 4)


,id_web,total_sales,product_name,post_type
0,bon-cadeau-25-euros,10.0,bon-cadeau-de-25-euros,product
1,15298,6.0,pierre-jean-villa-saint-joseph-preface-2018,product
2,15296,0.0,pierre-jean-villa-saint-joseph-tilde-2017,product
3,15300,0.0,pierre-jean-villa-croze-hermitage-accroche-coe...,product
4,19814,3.0,pierre-jean-villa-igp-gamine-2018,product


## premières statistiques tirées de la DF erp

Nous pouvons à présent utiliser à nouveau notre outil de profiling report pour obtenir les premières statistiques cette DF.

In [11]:
# profiling report pour obtenir les premières statistiques de la DF web

profileWeb = ProfileReport(web, title="Pandas Profiling Report")
profileWeb.to_widgets()

Generate report structure: 100%|██████████| 1/1 [00:01<00:00,  1.98s/it]


1. Overview: 714 lignes, 5 colonnes, 0 NaN, 0 duplicate rows
2. Variables:
    1. df_index: 714 valeurs distinctes
    2. id_web: pas d'informations particulières
    3. total_sales: valeurs nules ou positives comprises entre 0 et 96 avec une moyenne de 4.01
    4. product_name: 714 valeurs distinctes
    5. post_type: 714 fois la valeur "attachment"
3. interactions: nous permet de voir une représentation graphique des ventes par produit
4. missing value: pas de valeurs manquantes
5. sample: affichage des premières et dernières lignes de la DF




# 8 Variables et fonctions:<a class="anchor" id="chapter8"></a>

L'idée est de déclarer ici toutes les variables et fonctions qui seront utilisées pour notre analyse et pour répondre aux questions posées.
les questions posées correspondant exclusivement aux produits en ligne, nous éliminerons les produits non commercialisés sur le site.

In [12]:
# Variables et fonctions
# créations de bases
all_bases = {
    "erp" : erp,
    "liaisons" : liaison,
    "web" : web
}

# création d'une base erp_online qui ne contient que les lignes d'erp correspondant à des produits commercialisés en ligne.
erp_online = erp[erp.onsale_web == 1]


# jointures
# entre l'erp "en ligne" et le fichier de liaison:
erp_online_liaison = pd.merge(erp_online, liaison, how="outer", on="product_id")

# entre le fichier web et le fichier de liaison:
web_liaison = pd.merge(web, liaison, how="outer", on="id_web")

# entre l'erp "en ligne" et la jointure du fichier web et liaison:
total_base = pd.merge(erp_online, web_liaison, on="product_id")

# suppression des 3 NaN correspondant aux produits référencés dans erp_online et pas dans web
total_base = total_base[total_base.id_web.isna() == False]

# calcul de la valeur du stock
total_base["valeur_stock"] = total_base.stock_quantity * total_base.price

# chiffre d'affaire 
total_base["chiffre_affaire"] = total_base['price']*total_base['total_sales']

# valeur stock > CA
total_base["stockSupCA"] = total_base.valeur_stock > total_base.chiffre_affaire

# ajout d'un Booleen si le produit s'est déjà vendu
total_base["sold"] = total_base.chiffre_affaire > 0 

# Contrôles

print(f' product_id dans erp : {len(erp.product_id.unique())}')
print(f' product_id dans liaison : {len(liaison.product_id.unique())}')
print(f' id_web dans liaison : {len(liaison.id_web.unique())}')
print(f' id_web dans web : {len(web.id_web.unique())}')
print(f' product_id dans erp_online : {len(erp_online.product_id.unique())}')
print(f'forme de la df totale: {total_base.shape[0]} lignes, {total_base.shape[1]} colonnes')
total_base.head()

 product_id dans erp : 825
 product_id dans liaison : 825
 id_web dans liaison : 735
 id_web dans web : 715
 product_id dans erp_online : 717
forme de la df totale: 714 lignes, 12 colonnes


,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
0,3847,1,24.2,0,15298,6.0,pierre-jean-villa-saint-joseph-preface-2018,product,0.0,145.2,False,True
1,3849,1,34.3,0,15296,0.0,pierre-jean-villa-saint-joseph-tilde-2017,product,0.0,0.0,False,False
2,3850,1,20.8,0,15300,0.0,pierre-jean-villa-croze-hermitage-accroche-coe...,product,0.0,0.0,False,False
3,4032,1,14.1,0,19814,3.0,pierre-jean-villa-igp-gamine-2018,product,0.0,42.3,False,True
4,4039,1,46.0,0,19815,0.0,pierre-jean-villa-cote-rotie-carmina-2017,product,0.0,0.0,False,False



## 9 Analyse et mise en forme des données <a class="anchor" id="chapter9"></a>

On vérifie la pertinence des données recueillies et on les met en forme pour l'analyse demandée.



On constate que 825 produits sont listés dans l'ERP et dans le fichier de liaison.
on remarque que si on restreint l'erp aux produits commercialisés en ligne nous avons 717 références et seulement 714 références dans la df web, nous avons donc 3 références manquantes.

In [13]:
# produits vendus sur le web mais absents de la base de liaison 

erp_online_a_lier = erp_online_liaison[(erp_online_liaison.id_web.isna() == True) & (erp_online_liaison.onsale_web == 1)]
erp_online_a_lier


,product_id,onsale_web,price,stock_quantity,id_web
228,4594,1.0,144.0,0.0,NaN
449,5070,1.0,84.7,0.0,NaN
450,5075,1.0,43.3,0.0,NaN


Ces 3 produits référencés dans l'ERP comme commercialisés en ligne mais absents du site n'etant pas en stock, nous tirons la conclusion qu'ils seront référencés sur le site web au moment de leur remise en stock et nous choisissons de ne plus en tenir compte pour la suite de notre analyse.





## 10 premières statistiques globales<a class="anchor" id="chapter10"></a>

In [14]:
profiletotal = ProfileReport(total_base, title="Pandas Profiling Report")
profileWeb.to_widgets()

1. Overview : 714 lignes 5 colones, 0 NaN, 0 Duplicate
2. Variables:
    1. df_index: 714 distinct
    2. id_web: pas d'informations
    3. total sales: 41 tarifs distincts, uniquement des valeurs nulles (pas de vente) ou positives, comprises entre 0 et 96, avec une moyenne de 4.01, on notera que sur les 714 références, 329 (soit 46.1%) ne se sont jamais vendues.
    4. product_name: 714 noms distincts
    5. post type: 714 valeurs "attachment"
3. Interactions:
    1. graphique des ventes par index
4. correlations:
5. missing values: aucune
6. Sample: premières et dernières lignes de la DF

Il est envisageable de lister les produits invendus, ainsi que l'etat de leur stock pour questionner leur placement tarifaire ou disponibilité au catalogue.



In [15]:
# On vérifie que tous les produits référencés sur web aient un product_id

print(web_liaison.product_id.isna().unique())


[False]



#  11 Résolution des problèmes énoncés<a class="anchor" id="chapter11"></a>

Il nous a été demandé de rapprocher les bases entre elles, d'en déduire le CA total, le CA par produit et d'y rechercher des valeurs anormales




## 12 rapprochement des bases<a class="anchor" id="chapter12"></a>

On avait créé une DF en procédant à une jointure entre les 3 DF fournies dans la section "variables et fonctions".

In [16]:
# Résolution des problèmes énoncés
# On appelle une ligne au hazard pour avoir un apperçu des informations contenue dans la DF

total_base.iloc[228]

product_id                                                      4596
onsale_web                                                         1
price                                                           43.9
stock_quantity                                                     0
id_web                                                         15476
total_sales                                                       23
product_name       marc-colin-et-fils-chassagne-montrachet-blanc-...
post_type                                                    product
valeur_stock                                                       0
chiffre_affaire                                               1009.7
stockSupCA                                                     False
sold                                                            True
Name: 230, dtype: object


## 13 chiffre d'affaire total en  ligne <a class="anchor" id="chapter13"></a>

Comme nous avions retiré de la DF tous les produits non commercialisés en ligne, il nous suffit de faire la somme des valeurs de la colonne chiffre_affaire






In [17]:


# total du chiffre d'affaire réalisé en ligne
ca_total = total_base.chiffre_affaire.sum()
print(f"total du chiffre d'affaire en ligne : {ca_total} €")




total du chiffre d'affaire en ligne : 70568.6 €


Le chiffre d'affaire total de la boutique en ligne est de 70568.6€.

##  14 Par produit <a class="anchor" id="chapter14"></a>

le Chiffre d'affaire par produit est contenu dans la colonne chiffre_affaire pour chaque ligne.

In [18]:
# chiffre d'affaire par produit classé par CA décroissant.

total_base.sort_values("chiffre_affaire", ascending=False, inplace=True)
total_base.head()


,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
194,4334,1,49.0,0,7818,96.0,champagne-gosset-grand-blanc-de-blanc,product,0.0,4704.0,False,True
71,4144,1,49.0,11,1662,87.0,champagne-gosset-grand-rose,product,539.0,4263.0,False,True
218,4402,1,176.0,8,3510,13.0,cognac-frapin-vip-xo,product,1408.0,2288.0,False,True
70,4142,1,53.0,8,11641,30.0,champagne-gosset-grand-millesime-2006,product,424.0,1590.0,False,True
69,4141,1,39.0,1,304,40.0,gosset-champagne-grande-reserve,product,39.0,1560.0,False,True




nous ne tiendrons pas compte des produits non vendus pour l'étude du chiffre d'affaire par produit.

In [19]:
# statistiques rapides du chiffre d'affaire par produit
ca_positif = total_base[total_base.sold == True]
ca_positif.chiffre_affaire.describe()

count     385.000000
mean      183.295065
std       400.324073
min         6.500000
25%        38.600000
50%        81.600000
75%       164.400000
max      4704.000000
Name: chiffre_affaire, dtype: float64

La boutique en ligne à commercialisé avec succés 385 références sur 714.

La meilleure vente apporte 4704€ de CA, la moins bonne 6,50€.
Le CA produit moyen est de 183.30€



In [20]:
data = ca_positif
fig = px.histogram(data, title="répartition des CA produits",
y="chiffre_affaire" )
fig.show()

Ce graphique nous permet de constater que la majorité des produits (306/385) ont un CA compris entre 0 et 199€ 



# 15 Recherche de valeurs anormales <a class="anchor" id="chapter15"></a>



In [21]:
# récupération des statistiques basiques de la colone price 
total_base.price.describe(include='all')


count    714.000000
mean      32.493137
std       27.810525
min        5.200000
25%       14.100000
50%       23.550000
75%       42.175000
max      225.000000
Name: price, dtype: float64

In [22]:
# seconde solution pour récupérer les statistiques basiques individuellement
data = total_base.price
print(f'moyenne : {data.mean()}')
print(f'médiane : {data.median()}')
print(f'mode : {data.mode()}')
# ddof = 0 sur la population complète
# ddof = 1 sur un échantillon de la population
print(f'variance : {data.var(ddof=0)}')
print(f'ecart type : {data.std(ddof=0)}')
print(f'skewness : {data.skew()}')
print(f'kurtosis : {data.kurtosis()}')
q1 = np.percentile(data, 25)
q3 = np.percentile(data, 75)
interquartile = q3-q1
print(f'intervale interquartile : {interquartile}')

moyenne : 32.49313725490197
médiane : 23.55
mode : 0    19.0
dtype: float64
variance : 772.342067748668
ecart type : 27.791042941002917
skewness : 2.5809012630033696
kurtosis : 10.088392064977288
intervale interquartile : 28.075000000000003


## Visualisation des valeurs sous forme d'histogramme

calcul du coéfficient d'asymétrie (skewness) 


In [23]:
#asymétrie de pearson (par le mode):  (moyenne - mode)/ecart type

skew = ((data.mean()-data.mode())/(data.std(ddof=0)))
print(f"coefficient d'asymétrie : {skew}")

# deuxième coéfficient d'asymétrie de pearson (par la médiane): (3(moyenne - médiane))/ecart-type
pearson2 = 3*(data.mean()-data.median())/(data.std(ddof=0))

print(f" deuxieme coefficient d'asymétrie : {pearson2}")

coefficient d'asymétrie : 0    0.485521
dtype: float64
 deuxieme coefficient d'asymétrie : 0.9653978017903665


Un coéfficient d'asymétrie positif indique une distribution décalée à gauche de la médiane 



In [24]:
# + representation graphique en histogramme
fig = px.histogram(data, title="répartition des prix produits" )
fig.show()


Effectivement l'histogramme confirme une distribution des prix à droite de la médiane, et nous indique la présence de valeurs anormales que nous allons devoir extraire et contrôler.

## méthode de la "boite à moustaches"

La boite à moustache nous permet de visualiser quelques indicateurs de position du caractère étudié (médiane, quartiles, minimum et maximum).
Il permet de visualiser très rapidement d'éventuelles valeurs extrèmes.


In [25]:
# calcul minimum statistique

min_stat = q1 - (interquartile*1.5)
print(f'minimum statistique : {min_stat}')


# Calcul du maximum statistique
max_stat = q3 + (interquartile*1.5)
print(f'maximum statistique : {max_stat}')

# recherche des valeurs dont le prix est supérieur au maximum statistique
total_base_highValue = total_base[total_base['price'] > max_stat]
print(f'nombre de valeurs avec des prix supérieurs aux maximum statistique : {total_base_highValue.shape[0]}')



minimum statistique : -28.012500000000003
maximum statistique : 84.28750000000001
nombre de valeurs avec des prix supérieurs aux maximum statistique : 32


le minimum statistique etant négatif, alors que notre valeur minimale est positive nous indique que nous n'avons pas de valeurs extrèmes par le bas.

Le maximum statistique etant de 84.28, alors que notre valeur maximale est de 225 nous indique que nous avons quelques valeurs extrèmes (Outliers) qu'il conviendra de lister et contrôler

Nous avons créé une nouvelle DF total_base_highValue qui contient nos outliers à contrôler (32 lignes)

Représentation graphique boite à moustache

In [26]:
# + representation graphique en boite à moustache
fig = px.box(data, x="price", 
points="all",
 title=" boite à moustache répartition des prix produit")
fig.show()

Sur cette représentation graphique, les "outliers" sont mis en évidence. 

## Liste des références extrèmes




In [27]:
# affichage des valeurs à contrôler
total_base_highValue.sort_values("price", ascending=False, inplace=True)

# recherche de résultat de ventes positifs:

total_base_highValue_isSold = total_base_highValue[(total_base_highValue.sold == 1)]
print(total_base_highValue_isSold.shape[0])

# appercu des premières lignes
total_base_highValue.head()

10


<ipython-input-27-0e0598c5389d>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
199,4352,1,225.0,0,15940,5.0,champagne-egly-ouriet-grand-cru-millesime-2008,product,0.0,1125.0,False,True
428,5001,1,217.5,20,14581,0.0,david-duband-charmes-chambertin-grand-cru-2014,product,4350.0,0.0,True,False
593,5892,1,191.3,10,14983,3.0,coteaux-champenois-egly-ouriet-ambonnay-rouge-...,product,1913.0,573.9,True,True
218,4402,1,176.0,8,3510,13.0,cognac-frapin-vip-xo,product,1408.0,2288.0,False,True
559,5767,1,175.0,12,15185,0.0,camille-giroud-clos-de-vougeot-2016,product,2100.0,0.0,True,False


Le marché des alcools et spiritueux, et particulièrement pour les vins et champagnes est marqué par des disparités de prix importantes.

Après vérification rapide via une recherche sur google, les prix de ces produits (d'exception) sont tout à fait dans la moyenne des tarifs proposés par la concurrence.

De plus une bonne partie (1/3) d'entre eux à déjà été commercialisé avec succès, le placement tarifaire parait donc cohérent.

# 16 Autres réflexions<a class="anchor" id="chapter16"></a>

Pendant mon exploration de données préalable à la résolution des problèmes énoncés, ma curiosité à été stimulée par certaines interrogations ou constats.

J'ai donc décidé de pousser l'analyse un peu plus loin pour satisfaire cette curiosité.

## 17 Des invendus, pourquoi? <a class="anchor" id="chapter17"></a>

In [28]:
# liste des invendus
invendus = total_base[total_base.sold == 0]
#liste des vendus
vendus = ca_positif

# statistiques de base
invendus.price.describe()



count    329.000000
mean      39.411094
std       29.399851
min        5.200000
25%       18.200000
50%       29.900000
75%       52.900000
max      217.500000
Name: price, dtype: float64

In [29]:
vendus.price.describe()

count    385.000000
mean      26.581429
std       24.934530
min        5.700000
25%       12.500000
50%       19.000000
75%       31.700000
max      225.000000
Name: price, dtype: float64

In [30]:
# statistiques individuelles
data2 = invendus.price
print(f'moyenne : {data2.mean()}')
print(f'médiane : {data2.median()}')
print(f'mode : {data2.mode()}')
# ddof = 0 sur la population complète
# ddof = 1 sur un échantillon de la population
print(f'variance : {data2.var(ddof=0)}')
print(f'ecart type : {data2.std(ddof=0)}')
print(f'skewness : {data2.skew()}')
print(f'kurtosis : {data2.kurtosis()}')

moyenne : 39.41109422492397
médiane : 29.9
mode : 0    78.0
dtype: float64
variance : 861.7240182555595
ecart type : 29.35513614779464
skewness : 1.9028913657153843
kurtosis : 5.701990997667269


In [31]:
data3 = vendus.price
print(f'moyenne : {data3.mean()}')
print(f'médiane : {data3.median()}')
print(f'mode : {data3.mode()}')
# ddof = 0 sur la population complète
# ddof = 1 sur un échantillon de la population
print(f'variance : {data3.var(ddof=0)}')
print(f'ecart type : {data3.std(ddof=0)}')
print(f'skewness : {data3.skew()}')
print(f'kurtosis : {data3.kurtosis()}')

moyenne : 26.581428571428578
médiane : 19.0
mode : 0     9.3
1     9.9
2    19.0
dtype: float64
variance : 620.115882374768
ecart type : 24.902126061337977
skewness : 3.719462793465191
kurtosis : 20.190770175928705


In [32]:

# histogramme comparatif

df = total_base
fig = px.histogram(df, x="price", color="sold", title="Répartition des prix produits")
fig.show()

In [33]:

# boxplot comparatif

df = total_base
fig = px.box(df, x="price", y="sold", points="all", title="boite à moustache des prix produits")
fig.show()

In [34]:
invendus.sort_values("stock_quantity", ascending=False)

,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
241,4609,1,11.8,237,15145,0.0,francois-bergeret-hautes-cotes-de-beaune-rouge...,product,2796.6,0.0,True,False
334,4749,1,11.9,140,15315,0.0,domaine-de-montgilet-anjou-blanc-2016,product,1666.0,0.0,True,False
191,4304,1,8.1,133,15120,0.0,idylle-savoie-cruet-2018,product,1077.3,0.0,True,False
566,5777,1,5.7,132,14338,0.0,maurel-pays-d-oc-cabernet-sauvignon-2017,product,752.4,0.0,True,False
572,5797,1,17.2,123,15688,0.0,chateau-saransot-dupre-listrac-medoc-2016,product,2115.6,0.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
41,4079,1,37.0,0,13078,0.0,vieux-donjon-chateauneuf-2013,product,0.0,0.0,False,False
31,4069,1,60.0,0,11862,0.0,gilles-robin-hermitage-2012,product,0.0,0.0,False,False
5,4040,1,34.3,0,15303,0.0,pierre-jean-villa-saint-joseph-saut-ange-2018,product,0.0,0.0,False,False
505,5561,1,58.0,0,19820,0.0,tempier-bandol-migoua-2017,product,0.0,0.0,False,False


In [35]:
invendus_stock = invendus[invendus.stock_quantity != 0]
print(f"nombre de produits en stock et invendus : {invendus_stock.shape[0]}")

nombre de produits en stock et invendus : 256


Les produits vendus ont un tarif moins élevé que les produits invendus, la répartition des prix des produits vendus, comme invendus est relativement similaire à la répartition totale des produits.

Le tarif des produits joue donc un rôle mais n'est pas un facteur déterminant dans la décision d'achat.

Après une recherche rapide sur google j'ai pu constater que le placement tarifaire des produits invendus est dans la moyenne de la concurrence.

Il existe du stock pour une majorité (256/329) produits invendus, le stock n'est donc pas non plus un facteur déterminant dans les résultats de vente.

Les résultats de vente positive ou non de chaque produit n'etant pas déterminés uniquement par le tarif ou par l'etat du stock, l'explication est aussi à chercher dans les gouts, habitudes et préférences de la clientèle.




## 18 gestion des stocks<a class="anchor" id="chapter18"></a>



In [36]:
# récupération de la valeur totale du stock
valeur_totale_stock = total_base.valeur_stock.sum()
print(f"nous avons une valeur de stock de {round(valeur_totale_stock/1000,2)}K€ pour un chiffre d'affaire de {round(ca_total/1000,2)}K€")

nous avons une valeur de stock de 387.84K€ pour un chiffre d'affaire de 70.57K€


In [37]:
# répartition du stock et du CA/produit

x0 = total_base.valeur_stock
# Add 1 to shift the mean of the Gaussian distribution
x1 = total_base.chiffre_affaire

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, name="valeur théorique du stock"))
fig.add_trace(go.Histogram(x=x1, name="chiffre d'affaire"))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()


In [38]:
# combien de produits ont une valeur de stock moins élevée que leur CA?

normalStock = total_base[total_base.stockSupCA == False]
print(f"Nombre de produits dont le CA est supérieur au stock : {normalStock.shape[0]}")
normalStock.sort_values("stock_quantity", ascending=False)


Nombre de produits dont le CA est supérieur au stock : 188


,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
306,4706,1,16.8,23,15349,32.0,albert-mann-muscat-2018,product,386.4,537.6,False,True
90,4164,1,7.6,19,13453,20.0,pares-balta-penedes-mas-petit-2015,product,144.4,152.0,False,True
33,4071,1,33.2,16,15953,19.0,vaudieu-chateauneuf-2015,product,531.2,630.8,False,True
17,4053,1,44.3,16,13127,23.0,clos-du-mont-olivet-chateauneuf-du-pape-2012,product,708.8,1018.9,False,True
155,4250,1,19.5,14,16317,30.0,domaine-saint-denis-bourgogne-clos-coque-2018,product,273.0,585.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
387,4910,1,17.3,0,13809,4.0,antoine-marie-arena-vin-de-france-rouge-san-gi...,product,0.0,69.2,False,True
177,4277,1,24.8,0,14865,3.0,i-fabbri-chianti-classico-riserva-2015,product,0.0,74.4,False,True
667,6225,1,20.4,0,15162,4.0,le-pas-de-lescalette-herault-ze-cinsault-2017,product,0.0,81.6,False,True
377,4893,1,27.9,0,15808,3.0,jacqueson-rully-rouge-1er-cru-les-cloux-2018,product,0.0,83.7,False,True


In [39]:
# parmis ces produits, combien d'entre eux sont en stock?

notNullNormalStock = normalStock[normalStock.valeur_stock > 0]
notNullNormalStock.shape[0]

47

In [40]:
# récupération de la liste des produits dont la valeur du stock dépasse celle de leur CA
overStock = total_base[total_base.stockSupCA == True]
print(overStock.shape)
overStock

(526, 12)


,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
657,6206,1,25.2,120,16580,41.0,domaine-giudicelli-patrimonio-blanc-2019,product,3024.0,1033.2,True,True
30,4068,1,16.6,157,16416,62.0,gilles-robin-crozes-hermitage-papillon-2019,product,2606.2,1029.2,True,True
658,6207,1,25.2,363,16077,37.0,domaine-giudicelli-patrimonio-rouge-2016,product,9147.6,932.4,True,True
383,4904,1,137.0,13,14220,5.0,domaine-des-croix-corton-charlemagne-grand-cru...,product,1781.0,685.0,True,True
9,4045,1,42.6,66,16041,14.0,pierre-gaillard-cote-rotie-2018,product,2811.6,596.4,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
354,4791,1,13.6,24,14599,0.0,maurice-schoech-pinot-noir-piece-de-chene-2016,product,326.4,0.0,True,False
353,4790,1,11.1,43,15732,0.0,maurice-schoech-riesling-2018,product,477.3,0.0,True,False
350,4786,1,12.1,37,15881,0.0,maurice-schoech-gewurztraminer-2018,product,447.7,0.0,True,False
348,4784,1,28.5,10,15734,0.0,maurice-schoech-riesling-vendanges-tardives-2017,product,285.0,0.0,True,False


In [41]:
x0 = overStock.valeur_stock
# Add 1 to shift the mean of the Gaussian distribution
x1 = overStock.chiffre_affaire

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, name="valeur théorique du stock"))
fig.add_trace(go.Histogram(x=x1, name="chiffre d'affaire"))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [42]:
# filtre les produits dont le CA est > 0
soldOverStock = overStock[overStock.chiffre_affaire > 0]
print(soldOverStock.shape)
soldOverStock.sort_values("valeur_stock", ascending=False)

(270, 12)


,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
658,6207,1,25.2,363,16077,37.0,domaine-giudicelli-patrimonio-rouge-2016,product,9147.6,932.4,True,True
126,4208,1,7.6,578,16024,16.0,domaine-montrose-cotes-de-thongue-rose-2019,product,4392.8,121.6,True,True
100,4176,1,13.5,276,15629,9.0,hortus-pic-saint-loup-la-bergerie-2018,product,3726.0,121.5,True,True
657,6206,1,25.2,120,16580,41.0,domaine-giudicelli-patrimonio-blanc-2019,product,3024.0,1033.2,True,True
442,5047,1,22.5,129,531,13.0,champagne-petit-lebrun-fils-blanc-de-blancs-gr...,product,2902.5,292.5,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
342,4776,1,6.8,11,14302,7.0,chateau-de-la-liquiere-pays-dherault-blanc-a-m...,product,74.8,47.6,True,True
607,5914,1,36.0,2,14265,1.0,darnleys-london-dry-gin-spiced,product,72.0,36.0,True,True
443,5056,1,7.5,9,13531,1.0,domaine-de-montgilet-anjou-rouge-2016-2,product,67.5,7.5,True,True
401,4929,1,16.7,4,15927,2.0,domaine-la-croix-belle-cotes-de-thongue-rouge-...,product,66.8,33.4,True,True


In [43]:
unsoldOverStock = overStock[overStock.chiffre_affaire <= 0]
print(unsoldOverStock.shape)
unsoldOverStock.sort_values("stock_quantity", ascending=False)

(256, 12)


,product_id,onsale_web,price,stock_quantity,id_web,total_sales,product_name,post_type,valeur_stock,chiffre_affaire,stockSupCA,sold
241,4609,1,11.8,237,15145,0.0,francois-bergeret-hautes-cotes-de-beaune-rouge...,product,2796.6,0.0,True,False
334,4749,1,11.9,140,15315,0.0,domaine-de-montgilet-anjou-blanc-2016,product,1666.0,0.0,True,False
191,4304,1,8.1,133,15120,0.0,idylle-savoie-cruet-2018,product,1077.3,0.0,True,False
566,5777,1,5.7,132,14338,0.0,maurel-pays-d-oc-cabernet-sauvignon-2017,product,752.4,0.0,True,False
572,5797,1,17.2,123,15688,0.0,chateau-saransot-dupre-listrac-medoc-2016,product,2115.6,0.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
322,4725,1,23.4,1,16010,0.0,francois-baur-riesling-grand-cru-brand-clos-de...,product,23.4,0.0,True,False
148,4240,1,28.0,1,15436,0.0,domaine-augustin-collioure-rouge-adeodat-2017,product,28.0,0.0,True,False
238,4605,1,32.2,1,15148,0.0,catherine-et-claude-marechal-savigny-les-beaun...,product,32.2,0.0,True,False
170,4269,1,10.2,1,16244,0.0,serol-cote-roannaise-originelles-2019,product,10.2,0.0,True,False


In [44]:
fig = px.histogram(unsoldOverStock.valeur_stock, title="répartition valeur stock par produit invendu" )
fig.show()